In [ ]:
class Earnings_Quality(CustomFactor):
    inputs = [morningstar.cash_flow_statement.operating_cash_flow,
              EarningsSurprises.eps_act]
    window_length = 1
    def compute(self, today, assets, out, operating_cash_flow, eps_act):  
        out[:] = operating_cash_flow / eps_act

In [ ]:
class sharpe(CustomFactor):
    inputs = [USEquityPricing.close]
    window_length = 63
    
    def compute(self, today, assets, out, close):
        ret = np.divide(np.diff(close),close[:-1])
        out[:] = (np.mean(ret)/np.std(ret))

In [ ]:
class HurstExp(CustomFactor):  
    inputs = [USEquityPricing.open]  
    window_length = int(252*0.5)
    
    def Hurst(self, ts):   #Fast
        # Create a range of lag values  
        lags=np.arange(2,20)  
        # Calculate variance array of the lagged differences  
        tau = [np.sqrt(np.std(np.subtract(ts[lag:], ts[:-lag]))) for lag in lags
        n = len(lags)  
        x = np.log(lags)  
        y = np.log(tau)  
        poly = (n*(x*y).sum() - x.sum()*y.sum()) / (n*(x*x).sum() - x.sum()*x.sum())

        # Return the Hurst exponent
        hurst_exp = poly*2.0
        return hurst_exp

    def compute(self, today, assets, out,  OPEN):
        SERIES = np.log(np.nan_to_num(OPEN)) 
        hurst_exp_per_asset = map(self.Hurst, [SERIES[:,col_id].flatten() for col_id in np.arange(SERIES.shape[1])])  
        out[:] = np.nan_to_num(hurst_exp_per_asset)

In [ ]:
class efficiency_ratio(CustomFactor):    
    inputs = [USEquityPricing.close, USEquityPricing.high, USEquityPricing.low]   
    window_length = 252
     
    def compute(self, today, assets, out, close, high, low):
        lb = self.window_length
        e_r = np.zeros(len(assets), dtype=np.float64)
        a=np.array([high[1:(lb):1]-low[1:(lb):1],
                     abs(high[1:(lb):1]-close[0:(lb-1):1]),
                     abs(low[1:(lb):1]-close[0:(lb-1):1])])      
        b=a.T.max(axis=1)
        c=b.sum(axis=1)
        e_r=abs(close[-1]-close[0]) / c  
        out[:] = e_r        

In [ ]:
class AverageRangeRatio(CustomFactor):
    inputs = [USEquityPricing.close, USEquityPricing.open, USEquityPricing.high, USEquityPricing.low]
    def compute(self, today, assets, out, close, open, high, low):
        spread = high - low
        avg_spread = np.nanmean(spread, axis=0)
        daily_move = close - open
        abs_daily_move = np.absolute(daily_move)
        avg_abs_daily_move = np.nanmean(abs_daily_move, axis=0)
        out[:] = avg_abs_daily_move / avg_spread

In [ ]:
class Value(CustomFactor):
    inputs = [mstar.cash_flow_statement.operating_cash_flow,
              mstar.valuation.enterprise_value] 
    window_length = 1
    def compute(self, today, assets, out, ocf, ev):
        factor_df = pd.DataFrame(index=assets)
        factor_df["ocf"] = ocf[-1]
        factor_df["ev"] = ev[-1]
        out[:] = (factor_df['ocf'] / factor_df['ev'])

In [ ]:
class MomentumRanking(CustomFactor):
    inputs=[USEquityPricing.close]
    window_length=252
    def compute(self, today, assets, out, close):
        value_table = pd.DataFrame(index=assets)
        value_table['mom1'] = close[-1] / close[-20] - 1                   
        value_table['mom2'] = close[-1] / close[-60] - 1                   
        value_table['mom3'] = close[-1] / close[-125] - 1                    
        value_table['mom4'] = close[-1] / close[0] - 1   
        
        out[:] = value_table.rank(ascending = False).mean(axis=1) 

In [ ]:
class growthscore(CustomFactor):
            inputs = [Fundamentals.growth_score]
            window_length = 1
            def compute(self, today, assets, out, growth_score):
                out[:] = preprocess(growth_score[-1,:])

In [ ]:
class AcquirersMultiple_1period(CustomFactor):    
        inputs = [Fundamentals.enterprise_value,
                  Fundamentals.ebitda,
                  Fundamentals.capital_expenditure]   
        def compute(self, today, assets, out, enterprise_value, ebitda, capital_expenditure):
            out[:] = enterprise_value/(ebitda+capital_expenditure)

In [ ]:
class BullBearIntensity(CustomFactor):
    inputs = [stocktwits.bull_minus_bear]
    window_length = 3

    def compute(self, today, assets, out, bull_minus_bear):
        np.nanmean(bull_minus_bear, axis=0, out=out)
                
class PsychSignalMessages(CustomFactor):
    inputs = [stocktwits.bull_scored_messages, stocktwits.bear_scored_messages]
    window_length = 30
    
    def compute(self, today, assets, out, bull_msgs, bear_msgs):
        np.nanmean(bull_msgs + bear_msgs, axis=0, out=out)

In [ ]:
def make_factors():
    
        class atr_10days_percent_z(CustomFactor):  
            inputs = [USEquityPricing.high, USEquityPricing.low, USEquityPricing.close]  
            window_length = 11  
            def compute(self, today, assets, out, high, low, close):  
                range_low = np.maximum(high[1:], close[:-1]) 
                range_high = np.minimum(low[1:], close[:-1])  
                out[:] = z_score(np.mean(((range_high - range_low)/close[:-1])*100, axis=0))
                
        class Direction(CustomFactor):
            inputs = [USEquityPricing.open, USEquityPricing.close]
            window_length = 21
            def compute(self, today, assets, out, open, close):
                p = (close-open)/close
                out[:] = preprocess(np.nansum(-p,axis=0))
                
        class mean_rev(CustomFactor):   
            inputs = [USEquityPricing.high,USEquityPricing.low,USEquityPricing.close]
            window_length = 30
            def compute(self, today, assets, out, high, low, close):
            
                p = (high+low+close)/3

                m = len(close[0,:])
                n = len(close[:,0])
                
                b = np.zeros(m)
                a = np.zeros(m)
                
                for k in range(10,n+1):
                    price_rel = np.nanmean(p[-k:,:],axis=0)/p[-1,:]
                    wt = np.nansum(price_rel)
                    b += wt*price_rel
                    price_rel = 1.0/price_rel
                    wt = np.nansum(price_rel)
                    a += wt*price_rel
                
                out[:] = preprocess(b-a)
                
        class volatility(CustomFactor):
            inputs = [USEquityPricing.high, USEquityPricing.low, USEquityPricing.close, USEquityPricing.volume]
            window_length = 5
            def compute(self, today, assets, out, high, low, close, volume):
                vol = np.nansum(volume,axis=0)*np.nansum(np.absolute((high-low)/close),axis=0)
                out[:] = preprocess(-vol)
                
        class growthscore(CustomFactor):
            inputs = [Fundamentals.growth_score]
            window_length = 1
            def compute(self, today, assets, out, growth_score):
                out[:] = preprocess(growth_score[-1,:])
                
        class Trendline(CustomFactor):
            inputs = [USEquityPricing.close]
            window_length = 252
            
            _x = np.arange(window_length)
            _x_var = np.var(_x)

            def compute(self, today, assets, out, close):
            
                x_matrix = repeat_last_axis(
                (self.window_length - 1) / 2 - self._x,
                len(assets),
                )

                y_bar = np.nanmean(close, axis=0)
                y_bars = repeat_first_axis(y_bar, self.window_length)
                y_matrix = close - y_bars

                out[:] = preprocess(-np.divide(
                (x_matrix * y_matrix).sum(axis=0) / self._x_var,
                self.window_length
                ))
                
        class PsychSignalMessages(CustomFactor):
            inputs = [stocktwits.bull_scored_messages, stocktwits.bear_scored_messages]
            window_length = 30
    
            def compute(self, today, assets, out, bull_msgs, bear_msgs):
                out[:] = preprocess(np.nanmean(bull_msgs + bear_msgs, axis=0))
        
        class BullBearIntensity(CustomFactor):
            inputs = [stocktwits.bull_minus_bear]
            window_length = 5

            def compute(self, today, assets, out, bull_minus_bear):
                out[:] = np.nanmean(bull_minus_bear, axis=0)
                
        class Altman_Z(CustomFactor):
            inputs=[factset.Fundamentals.zscore_qf]
            window_length = 1
            def compute(self, today, assets, out, zscore_qf):
                out[:] = preprocess(zscore_qf[-1])
                
        class Quick_Ratio(CustomFactor):
            inputs=[factset.Fundamentals.quick_ratio_qf]
            window_length = 1
            def compute(self, today, assets, out, quick_ratio_qf):
                out[:] = preprocess(quick_ratio_qf[-1])
                
        class efficiency_ratio(CustomFactor):    
            inputs = [USEquityPricing.close, USEquityPricing.high, USEquityPricing.low]   
            window_length = 126
            def compute(self, today, assets, out, close, high, low):
                lb = self.window_length
                e_r = np.zeros(len(assets), dtype=np.float64)
                a=np.array([high[1:(lb):1]-low[1:(lb):1],
                     abs(high[1:(lb):1]-close[0:(lb-1):1]),
                     abs(low[1:(lb):1]-close[0:(lb-1):1])])      
                b=a.T.max(axis=1)
                c=b.sum(axis=1)
                e_r=abs(close[-1]-close[0]) / c  
                out[:] = e_r
                
        class CapEx_Vol(CustomFactor):
            inputs=[factset.Fundamentals.capex_assets_qf]
            window_length = 2*252
            def compute(self, today, assets, out, capex_assets):   
                out[:] = preprocess(-np.ptp(capex_assets,axis=0))
                
        class TEM(CustomFactor):
            inputs=[factset.Fundamentals.capex_qf_asof_date,factset.Fundamentals.capex_qf,factset.Fundamentals.assets]
            window_length = 390
            def compute(self, today, assets, out, asof_date, capex, total_assets):
                values = capex/total_assets
                values[np.isinf(values)] = np.nan
                out_temp = np.zeros_like(values[-1,:])
                for column_ix in range(asof_date.shape[1]):
                    _, unique_indices = np.unique(asof_date[:, column_ix], return_index=True)
                    quarterly_values = values[unique_indices, column_ix]
                    if len(quarterly_values) < 6:
                        quarterly_values = np.hstack([np.repeat([np.nan], 6 - len(quarterly_values)),quarterly_values,])
                    out_temp[column_ix] = np.std(quarterly_values[-6:])
                    out[:] = preprocess(-out_temp)
                
        
                
        return {
            'atr_10days_percent_z':    atr_10days_percent_z,
            'TEM':                     TEM,
            'CapEx_Vol':               CapEx_Vol,
            'efficiency_ratio':        efficiency_ratio,
            'Altman_Z':                Altman_Z,
            'Quick_Ratio':             Quick_Ratio,
            'Direction':               Direction,
            'mean_rev':                mean_rev,
            'volatility':              volatility,
            'GrowthScore':             growthscore,
            'Trendline':               Trendline,
            'PsychSignalMessages':     PsychSignalMessages,
            'BullBearIntensity':       BullBearIntensity,
        }

In [ ]:
class Net_Debt_Growth_3M(CustomFactor):
        inputs = [bs.total_debt, bs.cash_and_cash_equivalents]
        window_length = 92

        def compute(self, today, assets, out, debt, cash):
            out[:] = ((debt[-1] - cash[-1]) - (debt[0] - cash[0])) / \
                (debt[0] - cash[0])

In [ ]:
class Asset_Turnover(CustomFactor):
        inputs = [is_.total_revenue, bs.total_assets]
        window_length = 252

        def compute(self, today, assets, out, sales, tot_assets):
            out[:] = (sales[-1] * 4.) / \
                ((tot_assets[-1] + tot_assets[0]) / 2.)

In [ ]:
class Downside_Risk(CustomFactor):
        inputs = [Returns(window_length=2)]
        window_length = 252

        def compute(self, today, assets, out, rets):
            down_rets = np.where(rets < 0, rets, np.nan)
            out[:] = np.nanstd(down_rets, axis=0)

In [ ]:
class Price_Oscillator(CustomFactor):
        inputs = [USEquityPricing.close]
        window_length = 252
        def compute(self, today, assets, out, close):
            four_week_period = close[-20:]
            out[:] = (np.nanmean(four_week_period, axis=0) /
                      np.nanmean(close, axis=0)) - 1.

In [ ]:
def make_factors():
    
        class atr_10days_percent_z(CustomFactor):  
            inputs = [USEquityPricing.high, USEquityPricing.low, USEquityPricing.close]  
            window_length = 11  
            def compute(self, today, assets, out, high, low, close):  
                range_low = np.maximum(high[1:], close[:-1]) 
                range_high = np.minimum(low[1:], close[:-1])  
                out[:] = z_score(np.mean(((range_high - range_low)/close[:-1])*100, axis=0))
                
        class volatility(CustomFactor):
            inputs = [USEquityPricing.high, USEquityPricing.low, USEquityPricing.close, USEquityPricing.volume]
            window_length = 5
            def compute(self, today, assets, out, high, low, close, volume):
                vol = np.nansum(volume,axis=0)*np.nansum(np.absolute((high-low)/close),axis=0)
                out[:] = preprocess(-vol)
                
        class Trendline(CustomFactor):
            inputs = [USEquityPricing.close]
            window_length = 252
            
            _x = np.arange(window_length)
            _x_var = np.var(_x)

            def compute(self, today, assets, out, close):
            
                x_matrix = repeat_last_axis(
                (self.window_length - 1) / 2 - self._x,
                len(assets),
                )

                y_bar = np.nanmean(close, axis=0)
                y_bars = repeat_first_axis(y_bar, self.window_length)
                y_matrix = close - y_bars

                out[:] = preprocess(-np.divide(
                (x_matrix * y_matrix).sum(axis=0) / self._x_var,
                self.window_length
                ))
                
        class Altman_Z(CustomFactor):
            inputs=[factset.Fundamentals.zscore_qf]
            window_length = 1
            def compute(self, today, assets, out, zscore_qf):
                out[:] = preprocess(zscore_qf[-1])
                
        class Quick_Ratio(CustomFactor):
            inputs=[factset.Fundamentals.quick_ratio_qf]
            window_length = 1
            def compute(self, today, assets, out, quick_ratio_qf):
                out[:] = preprocess(quick_ratio_qf[-1])
                
        class CapEx_Vol(CustomFactor):
            inputs=[factset.Fundamentals.capex_assets_qf]
            window_length = 2*252
            def compute(self, today, assets, out, capex_assets):   
                out[:] = preprocess(-np.ptp(capex_assets,axis=0))
                
        class TEM(CustomFactor):
            inputs=[factset.Fundamentals.capex_qf_asof_date,factset.Fundamentals.capex_qf,factset.Fundamentals.assets]
            window_length = 390
            def compute(self, today, assets, out, asof_date, capex, total_assets):
                values = capex/total_assets
                values[np.isinf(values)] = np.nan
                out_temp = np.zeros_like(values[-1,:])
                for column_ix in range(asof_date.shape[1]):
                    _, unique_indices = np.unique(asof_date[:, column_ix], return_index=True)
                    quarterly_values = values[unique_indices, column_ix]
                    if len(quarterly_values) < 6:
                        quarterly_values = np.hstack([np.repeat([np.nan], 6 - len(quarterly_values)),quarterly_values,])
                    out_temp[column_ix] = np.std(quarterly_values[-6:])
                    out[:] = preprocess(-out_temp)
                
        
                
        return {
            'atr_10days_percent_z':    atr_10days_percent_z,
            'TEM':                     TEM,
            'CapEx_Vol':               CapEx_Vol,
            'Altman_Z':                Altman_Z,
            'Quick_Ratio':             Quick_Ratio,
            'volatility':              volatility,
            'Trendline':               Trendline,
        }

In [ ]:
class GP_to_A(CustomFactor):
    inputs = [morningstar.income_statement.gross_profit, morningstar.balance_sheet.total_assets]
    window_length = 1
    def compute(self, today, assets, out, gross_profit, total_assets):       
        out[:] = preprocess(gross_profit[-1] / total_assets[-1])

In [ ]:
class ItoA(CustomFactor):
    inputs = [factset.Fundamentals.ppe_gross,
              factset.Fundamentals.inven,
              factset.Fundamentals.assets]
    window_length = 270
    def compute(self, today, assets, out, ppe, inv, ta):
        ppe = np.nan_to_num(ppe)
        inv = np.nan_to_num(inv)
        out[:] = preprocess(-(ppe[-1]-ppe[0]+inv[-1]-inv[0])/ta[0])

In [ ]:
class ClenowMomentum(CustomFactor):
    inputs = [USEquityPricing.close]
    window_length = 90
    window_safe = True
    def compute(self, today, assets, out, close):
        res = []
        for i in range(close.shape[1]):
            res.append(_slope(close[:, i]))
        out[:] = res

In [ ]:
class EnterpriseValue(CustomFactor):  
    # Pre-declare inputs and window_length  
    inputs = [USEquityPricing.close, morningstar.valuation.shares_outstanding, morningstar.balance_sheet.long_term_debt,  
              morningstar.balance_sheet.long_term_debt_and_capital_lease_obligation,  
              morningstar.balance_sheet.cash_cash_equivalents_and_marketable_securities]  
    window_length = 1  
    # Compute market cap value  
    def compute(self, today, assets, out, close, shares,  
                long_term_debt, long_term_debt_and_capital_lease_obligation,  
                cash_cash_equivalents_and_marketable_securities):  
        out[:] = close[-1] * shares[-1] + long_term_debt_and_capital_lease_obligation[-1] - cash_cash_equivalents_and_marketable_securities[-1]

In [ ]:
class Sales(CustomFactor):   
    inputs = [morningstar.income_statement.operating_revenue]  
    window_length = 189
    def compute(self, today, assets, out, sales):       
        out[:] = sales[0]+sales[-1]+sales[-64]+sales[-127]

In [ ]:
def make_factors():
    
    class efficiency_ratio(CustomFactor):    
        inputs = [USEquityPricing.close, USEquityPricing.high, USEquityPricing.low]   
        window_length = 126
     
        def compute(self, today, assets, out, close, high, low):
            lb = self.window_length
            e_r = np.zeros(len(assets), dtype=np.float64)
            a=np.array([high[1:(lb):1]-low[1:(lb):1],
                     abs(high[1:(lb):1]-close[0:(lb-1):1]),
                     abs(low[1:(lb):1]-close[0:(lb-1):1])])      
            b=a.T.max(axis=1)
            c=b.sum(axis=1)
            e_r=abs(close[-1]-close[0]) / c  
            out[:] = preprocess(e_r)        
                
    class fcf(CustomFactor):
        inputs = [Fundamentals.fcf_yield]
        window_length = 1
        def compute(self, today, assets, out, fcf_yield):
            out[:] = preprocess(np.nan_to_num(fcf_yield[-1,:]))
                
    class Trendline(CustomFactor):
        inputs = [USEquityPricing.close]
        window_length = 252
            
        _x = np.arange(window_length)
        _x_var = np.var(_x)
 
        def compute(self, today, assets, out, close):
            
            x_matrix = repeat_last_axis(
            (self.window_length - 1) / 2 - self._x,
            len(assets),
            )
 
            y_bar = np.nanmean(close, axis=0)
            y_bars = repeat_first_axis(y_bar, self.window_length)
            y_matrix = close - y_bars
 
            out[:] = preprocess(-np.divide(
            (x_matrix * y_matrix).sum(axis=0) / self._x_var,
            self.window_length
            ))
                
    class SalesGrowth(CustomFactor):
        inputs = [factset.Fundamentals.sales_gr_qf]
        window_length = 2*252
        def compute(self, today, assets, out, sales_growth):
            sales_growth = np.nan_to_num(sales_growth)
            sales_growth = preprocessing.scale(sales_growth,axis=0)
            out[:] = preprocess(sales_growth[-1])
                
    class fcf_ev(CustomFactor):
        inputs=[
            Fundamentals.fcf_per_share,
            Fundamentals.shares_outstanding,
            Fundamentals.enterprise_value,]
        window_length = 1
        def compute(self, today, assets, out, fcf, shares, ev):
            v = fcf*shares/ev
            v[np.isinf(v)] = np.nan
                 
            out[:] = preprocess(v[-1])
                
    class AdvancedMomentum(CustomFactor):
        inputs = (USEquityPricing.close, Returns(window_length=126))
        window_length = 252
 
        def compute(self, today, assets, out, prices, returns):
            am = np.divide(
            (
            (prices[-21] - prices[-252]) / prices[-252] -
            prices[-1] - prices[-21]
            ) / prices[-21],
            np.nanstd(returns, axis=0)
            )
                
            out[:] = preprocess(-am)
            
    class Value(CustomFactor):
        # value factor based on OCF/EV    
        inputs = [cfs.operating_cash_flow, v.enterprise_value] 
        window_length = 1
  
        def compute(self, today, assets, out, ocf, ev):
            factor_df = pd.DataFrame(index=assets)
            factor_df["ocf"] = ocf[-1]
            factor_df["ev"] = ev[-1]
            out[:] = preprocess((factor_df['ocf'] / factor_df['ev']))
            
    class GP_to_A(CustomFactor):
        inputs = [is_.gross_profit, bs.total_assets]
        window_length = 1
        def compute(self, today, assets, out, gross_profit, total_assets):       
            out[:] = preprocess(gross_profit[-1] / total_assets[-1])
            
    class ItoA(CustomFactor):
        inputs = [factset.Fundamentals.ppe_gross,
              factset.Fundamentals.inven,
              factset.Fundamentals.assets]
        window_length = 270

        def compute(self, today, assets, out, ppe, inv, ta):
            ppe = np.nan_to_num(ppe)
            inv = np.nan_to_num(inv)
            out[:] = preprocess(-(ppe[-1]-ppe[0]+inv[-1]-inv[0])/ta[0])
                
    class CapEx_Vol(CustomFactor):
        inputs=[factset.Fundamentals.capex_assets_qf]
        window_length = 2*252
        def compute(self, today, assets, out, capex_assets):   
            out[:] = preprocess(-np.ptp(capex_assets,axis=0))
                
    class Asset_Turnover(CustomFactor):
        inputs = [is_.total_revenue, bs.total_assets]
        window_length = 252
        def compute(self, today, assets, out, sales, tot_assets):
            out[:] = preprocess((sales[-1] * 4.) / \
                ((tot_assets[-1] + tot_assets[0]) / 2.))
                
    class Price_Oscillator(CustomFactor):
        inputs = [USEquityPricing.close]
        window_length = 252
        def compute(self, today, assets, out, close):
            four_week_period = close[-20:]
            out[:] = preprocess((np.nanmean(four_week_period, axis=0) /
                np.nanmean(close, axis=0)) - 1.)
        
    class volatility(CustomFactor):
        inputs = [USEquityPricing.high, USEquityPricing.low, USEquityPricing.close, USEquityPricing.volume]
        window_length = 5
        def compute(self, today, assets, out, high, low, close, volume):
            vol = np.nansum(volume,axis=0)*np.nansum(np.absolute((high-low)/close),axis=0)
            out[:] = preprocess(-vol)
            
    class TEM(CustomFactor):
        inputs=[factset.Fundamentals.capex_qf_asof_date,factset.Fundamentals.capex_qf,factset.Fundamentals.assets]
        window_length = 390
        def compute(self, today, assets, out, asof_date, capex, total_assets):
            values = capex/total_assets
            values[np.isinf(values)] = np.nan
            out_temp = np.zeros_like(values[-1,:])
            for column_ix in range(asof_date.shape[1]):
                _, unique_indices = np.unique(asof_date[:, column_ix], return_index=True)
                quarterly_values = values[unique_indices, column_ix]
                if len(quarterly_values) < 6:
                    quarterly_values = np.hstack([np.repeat([np.nan], 6 - len(quarterly_values)),quarterly_values,])
                out_temp[column_ix] = np.std(quarterly_values[-6:])
                out[:] = preprocess(-out_temp)
                
    class ClenowMomentum(CustomFactor):
        inputs = [USEquityPricing.close]
        window_length = 90
        window_safe = True
        def compute(self, today, assets, out, close):
            res = []
            for i in range(close.shape[1]):
                res.append(_slope(close[:, i]))
            out[:] = preprocess(res)
            
    class Sales(CustomFactor):   
        inputs = [morningstar.income_statement.operating_revenue]  
        window_length = 189
        def compute(self, today, assets, out, sales):       
            out[:] = preprocess(sales[0]+sales[-1]+sales[-64]+sales[-127])
            
    class EnterpriseValue(CustomFactor):  
        inputs = [USEquityPricing.close, morningstar.valuation.shares_outstanding, morningstar.balance_sheet.long_term_debt,  
              morningstar.balance_sheet.long_term_debt_and_capital_lease_obligation,  
              morningstar.balance_sheet.cash_cash_equivalents_and_marketable_securities]  
        window_length = 1  
        def compute(self, today, assets, out, close, shares,  
                long_term_debt, long_term_debt_and_capital_lease_obligation,  
                cash_cash_equivalents_and_marketable_securities):  
            out[:] = preprocess(close[-1] * shares[-1] + long_term_debt_and_capital_lease_obligation[-1] - cash_cash_equivalents_and_marketable_securities[-1])
        
    return {
            'ClenowMomentum':          ClenowMomentum,
            'EnterpriseValue':         EnterpriseValue,
            'Sales':                   Sales,
            'CapEx_Vol':               CapEx_Vol,
            'Asset_Turnover':          Asset_Turnover,
            'Price_Oscillator':        Price_Oscillator,
            'volatility5':             volatility,
            'TEM':                     TEM,
            'ItoA':                    ItoA,
            'GP_to_A':                 GP_to_A,
            'Value':                   Value,
            'efficiency_ratio':        efficiency_ratio,
            'FCF':                     fcf,
            'Trendline':               Trendline,
            'SalesGrowth':             SalesGrowth,
            'fcf_ev':                  fcf_ev,
            'AdvancedMomentum':        AdvancedMomentum,
        }

In [ ]:
def make_factors():
    
    class efficiency_ratio(CustomFactor):    
        inputs = [USEquityPricing.close, USEquityPricing.high, USEquityPricing.low]   
        window_length = 126
     
        def compute(self, today, assets, out, close, high, low):
            lb = self.window_length
            e_r = np.zeros(len(assets), dtype=np.float64)
            a=np.array([high[1:(lb):1]-low[1:(lb):1],
                     abs(high[1:(lb):1]-close[0:(lb-1):1]),
                     abs(low[1:(lb):1]-close[0:(lb-1):1])])      
            b=a.T.max(axis=1)
            c=b.sum(axis=1)
            e_r=abs(close[-1]-close[0]) / c  
            out[:] = preprocess(e_r)        
                
    class fcf(CustomFactor):
        inputs = [Fundamentals.fcf_yield]
        window_length = 1
        def compute(self, today, assets, out, fcf_yield):
            out[:] = preprocess(np.nan_to_num(fcf_yield[-1,:]))
                
    class Trendline(CustomFactor):
        inputs = [USEquityPricing.close]
        window_length = 252
            
        _x = np.arange(window_length)
        _x_var = np.var(_x)
 
        def compute(self, today, assets, out, close):
            
            x_matrix = repeat_last_axis(
            (self.window_length - 1) / 2 - self._x,
            len(assets),
            )
 
            y_bar = np.nanmean(close, axis=0)
            y_bars = repeat_first_axis(y_bar, self.window_length)
            y_matrix = close - y_bars
 
            out[:] = preprocess(-np.divide(
            (x_matrix * y_matrix).sum(axis=0) / self._x_var,
            self.window_length
            ))
                
    class SalesGrowth(CustomFactor):
        inputs = [factset.Fundamentals.sales_gr_qf]
        window_length = 2*252
        def compute(self, today, assets, out, sales_growth):
            sales_growth = np.nan_to_num(sales_growth)
            sales_growth = preprocessing.scale(sales_growth,axis=0)
            out[:] = preprocess(sales_growth[-1])
                
    class fcf_ev(CustomFactor):
        inputs=[
            Fundamentals.fcf_per_share,
            Fundamentals.shares_outstanding,
            Fundamentals.enterprise_value,]
        window_length = 1
        def compute(self, today, assets, out, fcf, shares, ev):
            v = fcf*shares/ev
            v[np.isinf(v)] = np.nan
                 
            out[:] = preprocess(v[-1])
                
    class AdvancedMomentum(CustomFactor):
        inputs = (USEquityPricing.close, Returns(window_length=126))
        window_length = 252
 
        def compute(self, today, assets, out, prices, returns):
            am = np.divide(
            (
            (prices[-21] - prices[-252]) / prices[-252] -
            prices[-1] - prices[-21]
            ) / prices[-21],
            np.nanstd(returns, axis=0)
            )
                
            out[:] = preprocess(-am)
            
    class Value(CustomFactor):
        # value factor based on OCF/EV    
        inputs = [cfs.operating_cash_flow, v.enterprise_value] 
        window_length = 1
  
        def compute(self, today, assets, out, ocf, ev):
            factor_df = pd.DataFrame(index=assets)
            factor_df["ocf"] = ocf[-1]
            factor_df["ev"] = ev[-1]
            out[:] = preprocess((factor_df['ocf'] / factor_df['ev']))
            
    class GP_to_A(CustomFactor):
        inputs = [is_.gross_profit, bs.total_assets]
        window_length = 1
        def compute(self, today, assets, out, gross_profit, total_assets):       
            out[:] = preprocess(gross_profit[-1] / total_assets[-1])
            
    class ItoA(CustomFactor):
        inputs = [factset.Fundamentals.ppe_gross,
              factset.Fundamentals.inven,
              factset.Fundamentals.assets]
        window_length = 270

        def compute(self, today, assets, out, ppe, inv, ta):
            ppe = np.nan_to_num(ppe)
            inv = np.nan_to_num(inv)
            out[:] = preprocess(-(ppe[-1]-ppe[0]+inv[-1]-inv[0])/ta[0])
                
    class CapEx_Vol(CustomFactor):
        inputs=[factset.Fundamentals.capex_assets_qf]
        window_length = 2*252
        def compute(self, today, assets, out, capex_assets):   
            out[:] = preprocess(-np.ptp(capex_assets,axis=0))
        
    class volatility(CustomFactor):
        inputs = [USEquityPricing.high, USEquityPricing.low, USEquityPricing.close, USEquityPricing.volume]
        window_length = 5
        def compute(self, today, assets, out, high, low, close, volume):
            vol = np.nansum(volume,axis=0)*np.nansum(np.absolute((high-low)/close),axis=0)
            out[:] = preprocess(-vol)
            
    class TEM(CustomFactor):
        inputs=[factset.Fundamentals.capex_qf_asof_date,factset.Fundamentals.capex_qf,factset.Fundamentals.assets]
        window_length = 390
        def compute(self, today, assets, out, asof_date, capex, total_assets):
            values = capex/total_assets
            values[np.isinf(values)] = np.nan
            out_temp = np.zeros_like(values[-1,:])
            for column_ix in range(asof_date.shape[1]):
                _, unique_indices = np.unique(asof_date[:, column_ix], return_index=True)
                quarterly_values = values[unique_indices, column_ix]
                if len(quarterly_values) < 6:
                    quarterly_values = np.hstack([np.repeat([np.nan], 6 - len(quarterly_values)),quarterly_values,])
                out_temp[column_ix] = np.std(quarterly_values[-6:])
                out[:] = preprocess(-out_temp)
                
    class ClenowMomentum(CustomFactor):
        inputs = [USEquityPricing.close]
        window_length = 90
        window_safe = True
        def compute(self, today, assets, out, close):
            res = []
            for i in range(close.shape[1]):
                res.append(_slope(close[:, i]))
            out[:] = preprocess(res)
            
    class Sales(CustomFactor):   
        inputs = [morningstar.income_statement.operating_revenue]  
        window_length = 189
        def compute(self, today, assets, out, sales):       
            out[:] = preprocess(sales[0]+sales[-1]+sales[-64]+sales[-127])
        
    return {
            'ClenowMomentum':          ClenowMomentum,
            'Sales':                   Sales,
            'CapEx_Vol':               CapEx_Vol,
            'volatility5':             volatility,
            'TEM':                     TEM,
            'ItoA':                    ItoA,
            'GP_to_A':                 GP_to_A,
            'Value':                   Value,
            'efficiency_ratio':        efficiency_ratio,
            'FCF':                     fcf,
            'Trendline':               Trendline,
            'SalesGrowth':             SalesGrowth,
            'fcf_ev':                  fcf_ev,
            'AdvancedMomentum':        AdvancedMomentum,
        }

In [ ]:
    class AverageRangeRatio(CustomFactor):
            inputs = [USEquityPricing.close, USEquityPricing.open, USEquityPricing.high, USEquityPricing.low]
            window_length = 63
            def compute(self, today, assets, out, close, open, high, low):
                spread = high - low
                avg_spread = np.nanmean(spread, axis=0)
                daily_move = close - open
                abs_daily_move = np.absolute(daily_move)
                avg_abs_daily_move = np.nanmean(abs_daily_move, axis=0)
                out[:] = preprocess(avg_abs_daily_move / avg_spread)

In [ ]:
class RSI(CustomFactor):
        window_length = 15
        inputs = (USEquityPricing.close,)
 
        def compute(self, today, assets, out, closes):
            diffs = np.diff(closes, axis=0)
            ups = np.nanmean(np.clip(diffs, 0, np.inf), axis=0)
            downs = abs(np.nanmean(np.clip(diffs, -np.inf, 0), axis=0))
            rs = ups/downs
            rsi = 100.0-100.0/(1.0+rs)
            out[:] = preprocess(rsi)

In [ ]:
class accern_sentdex(CustomFactor):
    inputs = [sentiment_module.sentiment_signal,
             alphaone.article_sentiment]
    window_length = 20
    
    def compute(self,today,assets,out,sentdex,accern):
        #constants for scaling
        out[:] = (sentdex[-1]-1.5) + 5*accern[-1]

In [ ]:
class OBV_(CustomFactor):
        inputs = [USEquityPricing.close, USEquityPricing.volume]  
        window_length = 2  
        def compute(self, today, assets, out, close, vol):  
            diff = close[-1] - close[-2]  
            diff_abs = np.absolute(diff)  
            price_dir = diff/diff_abs  
            obv = pd.Series((price_dir * vol)[-1], index=assets)  
            if(context.obv is None):  
                context.obv = obv  
                context.running_obv = obv  
            else:  
                context.obv = context.obv.add(obv, fill_value=0)  
                context.running_obv.append(obv)  
            out[:] = context.obv[assets] 

In [ ]:
    
    class asset_to_equity_ratio(CustomFactor):
        inputs = [Fundamentals.total_assets, Fundamentals.common_stock_equity]
        window_length = 1
        
        def compute(self, today, assets, out, total_assets, common_stock_equity):
            out[:] = preprocess(total_assets[-1,:] / common_stock_equity[-1,:])
 
    class capex_to_cashflows(CustomFactor):
        inputs = [Fundamentals.capital_expenditure, Fundamentals.free_cash_flow]
        window_length = 1
        
        def compute(self, today, assets, out, capital_expenditure, free_cash_flow):
            out[:] = preprocess(capital_expenditure[-1,:] / free_cash_flow[-1,:])
    
    class ebitda_yield(CustomFactor):
        inputs = [Fundamentals.ebitda, USEquityPricing.close]
        window_length = 1
        
        def compute(self, today, assets, out, ebitda, close):
            out[:] = preprocess((ebitda[-1,:] * 4) / close[-1,:])
    
    class ebita_to_assets(CustomFactor):
        inputs = [Fundamentals.ebit, Fundamentals.total_assets]
        window_length = 1
        
        def compute(self, today, assets, out, ebit, total_assets):
            out[:] = preprocess((ebit[-1,:] * 4) / total_assets[-1,:])
    
    class operating_cashflows_to_assets(CustomFactor):
        inputs = [Fundamentals.operating_cash_flow, Fundamentals.total_assets]
        window_length = 1
        
        def compute(self, today, assets, out, operating_cash_flow, total_assets):
            out[:] = preprocess((operating_cash_flow[-1,:] * 4) / total_assets[-1,:])
    
    class MACD(CustomFactor):
        inputs = [USEquityPricing.close]
        window_length = 60
        def ema(self, data, window):      # Initial value for EMA is taken as trialing SMA
            c = 2.0 / (window + 1)
            ema = np.mean(data[-(2*window)+1:-window+1], axis=0)
            for value in data[-window+1:]:
                ema = (c * value) + ((1 - c) * ema)
            return ema
        def compute(self, today, assets, out, close):
            close = nanfill(close)
            fema = self.ema(close, 12)    # fast
            sema = self.ema(close, 26)    # slow
            macd_line = fema - sema
            macd = []
            macd.insert(0, self.ema(close,12) - self.ema(close,26))
            for i in range(1, 15, 1):
                macd.insert(0, self.ema(close[:-i],12) - self.ema(close[:-i],26))
            signal = self.ema(macd,9)
            out[:] = macd_line - signal
        
    return {
        'Asset to Equity Ratio'         : asset_to_equity_ratio,
        'Capex to Cashflows'            : capex_to_cashflows,
        'EBIT to Assets'                : ebita_to_assets,
        'EBITDA Yield'                  : ebitda_yield,
        'Operating Cashflows to Assets' : operating_cashflows_to_assets,
        'MACD'                          : MACD,
    }


In [ ]:
class AcquirersMultiple_1period(CustomFactor):    
        inputs = [Fundamentals.enterprise_value,
                  Fundamentals.ebitda,
                  Fundamentals.capital_expenditure]
        window_length = 1
        def compute(self, today, assets, out, enterprise_value, ebitda, capital_expenditure):
            am = enterprise_value/(ebitda+capital_expenditure)
            out[:] = preprocess(am[-1])

In [ ]:
class Yield(CustomFactor):  
    inputs = [morningstar.valuation_ratios.total_yield]  
    window_length = 1  
    def compute(self, today, assets, out, syield):  
        out[:] =  syield[-1]

In [ ]:
class pb_ratio(CustomFactor):
    inputs = [vr.pb_ratio.latest]
    window_length = 1 
    def compute(self, today, assets, out, pbr):
        out[:] = preprocess(pbr[-1])

In [ ]:
class bps_gr(CustomFactor):
    inputs = [factset.Fundamentals.bps_gr_af]
    window_length = 1 
    def compute(self, today, assets, out, bvy):
        out[:] = preprocess(bvy[-1])

In [ ]:
class Mean(CustomFactor):
    inputs = [USEquityPricing.close]
    window_length = 63
    def compute(self, today, asset_ids, out, close):
        r = np.diff(close,axis=0)/close[:-1]        
        out[:] = np.mean(r,axis=0)
        
class Variance(CustomFactor):
    inputs = [USEquityPricing.close]
    window_length = 252
    def compute(self, today, asset_ids, out, close):
        r = np.diff(close,axis=0)/close[:-1]        
        out[:] = np.var(r,axis=0)  

class UlcerIndex(CustomFactor):
    inputs = [USEquityPricing.close]
         window_length = 21
    def compute(self, today, asset_ids, out, close):
        max = np.maximum.accumulate(close,axis=0)
        dd = (max-close)/max
        dd = dd**2.0
        ui = np.sqrt(np.mean(dd,axis=0))
        out[:] = ui
        
class Value(CustomFactor):
    inputs = [morningstar.valuation_ratios.ev_to_ebitda]
    window_length = 1
    def compute(self, today, asset_ids, out, ratio):
        out[:] = 1.0/ratio[-1]
        
class Quality(CustomFactor):
    inputs = [morningstar.operation_ratios.roic]
    window_length = 1
    def compute(self, today, asset_ids, out, roic):
        out[:] = roic[-1]
        
class Growth_eps(CustomFactor):
        inputs = [morningstar.earnings_ratios.diluted_eps_growth]
        window_length = 1
        def compute(self, today, asset_ids, out, epsg):
            out[:] = preprocess(epsg[-1])
     
class pb_ratio(CustomFactor):
        inputs = [vr.pb_ratio.latest]
        window_length = 1 
        def compute(self, today, assets, out, pbr):
            out[:] = preprocess(pbr[-1])       
            
class Volume(CustomFactor):
    inputs = [USEquityPricing.volume]
    window_length = 1
    def compute(self, today, asset_ids, out, vol):
        out[:] = np.nan_to_num(np.mean(vol,axis=0))

In [ ]:
class Stochastic_Oscillator(CustomFactor): 
        inputs = [USEquityPricing.close,
                  USEquityPricing.high, USEquityPricing.low]
        window_length = 30
        def compute(self, today, assets, out, close, high, low):
            stoch_list = []
            for col_c, col_h, col_l in zip(close.T, high.T, low.T):
                try:
                    _, slowd = talib.STOCH(col_h, col_l, col_c,
                                           fastk_period=5, slowk_period=3, slowk_matype=0,
                                           slowd_period=3, slowd_matype=0)
                    stoch_list.append(slowd[-1])
                except:
                    stoch_list.append(np.nan)
            out[:] = preprocess(stoch_list)

In [ ]:
class VIXFactor(CustomFactor):  
        window_length = 1  
        inputs = [cboe_vix.vix_close]
        def compute(self, today, assets, out, vix):  
            out[:] = vix

In [ ]:
class SignedGapPct(CustomFactor):
        inputs = [USEquityPricing.close, USEquityPricing.open]
        window_length = 2
        def compute(self, today, asset_ids, out, close, open):
            out[:] = preprocess((open[-1] - close[-2]) / close[-2])

In [ ]:
from quantopian.interactive.data.quandl import cboe_vix
from quantopian.pipeline.data.quandl import cboe_vix

In [ ]:
class BETA(CustomFactor):
        inputs=[USEquityPricing.close]
        window_length = 6

        def compute(self, today, assets, out, close):

            # get returns dataset
            returns = ((close - np.roll(close, 1, axis=0)) / np.roll(close, 1, axis=0))[1:]

            # get index of benchmark
            benchmark_index = np.where((assets == 8554) == True)[0][0]

            # get returns of benchmark
            benchmark_returns = returns[:, benchmark_index]

            # prepare X matrix (x_is - x_bar)
            X = benchmark_returns
            X_bar = np.nanmean(X)
            X_vector = X - X_bar
            X_matrix = np.tile(X_vector, (len(returns.T), 1)).T

            # prepare Y matrix (y_is - y_bar)
            Y_bar = np.nanmean(close, axis=0)
            Y_bars = np.tile(Y_bar, (len(returns), 1))
            Y_matrix = returns - Y_bars

            # prepare variance of X
            X_var = np.nanvar(X)

            # multiply X matrix an Y matrix and sum (dot product)
            # then divide by variance of X
            # this gives the MLE of Beta
            out[:] = (np.sum((X_matrix * Y_matrix), axis=0) / X_var) / (len(returns))

In [ ]:
class Value(CustomFactor):
    # value factor based on OCF/EV    
    inputs = [mstar.cash_flow_statement.operating_cash_flow,
              mstar.valuation.enterprise_value] 
    window_length = 1
    def compute(self, today, assets, out, ocf, ev):
        factor_df = pd.DataFrame(index=assets)
        factor_df["ocf"] = ocf[-1]
        factor_df["ev"] = ev[-1]
        out[:] = (factor_df['ocf'] / factor_df['ev'])

In [ ]:
class fcf(CustomFactor):
        inputs = [Fundamentals.fcf_yield]
        window_length = 1
        window_safe = True
        def compute(self, today, assets, out, fcf_yield):
            out[:] = preprocess(np.nan_to_num(fcf_yield[-1,:]))

In [ ]:
class fcf_ch(CustomFactor):
        inputs = [Fundamentals.fcf_yield]
        window_length = 63
        window_safe = True
        def compute(self, today, assets, out, fcf_yield):
            out[:] = (fcf_yield[-63] - fcf_yield[-1]) / fcf_yield[-63]

In [ ]:
class volume_1dch(CustomFactor):
        inputs = [USEquityPricing.volume]
        window_length = 2
        window_safe = True
        def compute(self, today, assets, out, volume):
            out[:] = (volume[-1] - volume[-2]) / volume[-2]

In [ ]:
class fcf_ev_ch(CustomFactor):
        inputs=[
            Fundamentals.fcf_per_share,
            Fundamentals.shares_outstanding,
            Fundamentals.enterprise_value,]
        window_length = 63
        window_safe = True
        def compute(self, today, assets, out, fcf, shares, ev):
            q0 = fcf[-1]*shares[-1]/ev[-1]
            q0[np.isinf(q0)] = np.nan
            q1 = fcf[-63]*shares[-63]/ev[-63]
            q1[np.isinf(q1)] = np.nan
            out[:] = (q0 - q1) / q1

In [ ]:
class SMA_return(CustomFactor):
    inputs = [USEquityPricing.close]
        window_length = 21
        window_safe = True
    def compute(self, today, asset_ids, out, LongAvg, ShortAvg):
        ShortAvg = SimpleMovingAverage(
            inputs=[USEquityPricing.close],
            window_length=12,)
        LongAvg = SimpleMovingAverage(
            inputs=[USEquityPricing.close],
            window_length=50,)
        out[:] = (ShortAvg - LongAvg) / LongAvg

In [ ]:
class PctChange(CustomFactor):
    def compute(self, today, asset_ids, out, ShortAvg, LongAvg):
        out[:] = (values[-1] - values[0]) / values[0]

class Difference(CustomFactor):
    def compute(self, today, asset_ids, out, values):
        out[:] = values[-1] - values[0]